# this is a draft 

While writing the other code, it started to get a bit messy and difficult to understand 
this is a draft of a batch processing class which abstracts the processes of async, saving
and recovering from a failure.

It is not complete. Use the other functional method in the other notebook. 



In [2]:
import asyncio
import pickle
import logging
from typing import List, Callable, Any, Optional
import random
import nest_asyncio
from tqdm.asyncio import tqdm
import sys

nest_asyncio.apply()

# Disable all logging to console
logging.getLogger().handlers = []

class BatchProcessor:
    def __init__(
        self,
        process_func: Callable[[str], Any],
        batch_size: int = 100,
        concurrency_limit: int = 10,
        pickle_file: str = 'progress.pickle',
        logfile: Optional[str] = None
    ):
        self.process_func = process_func
        self.batch_size = batch_size
        self.concurrency_limit = concurrency_limit
        self.pickle_file = pickle_file
        self.semaphore = asyncio.Semaphore(concurrency_limit)
        self.processed_strings = []
        self.results = []

        # Set up logging
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        self.logger.handlers = []  # Clear any existing handlers
        formatter = logging.Formatter('%(asctime)s - %(message)s')

        # File handler (if logfile is provided)
        if logfile:
            file_handler = logging.FileHandler(logfile)
            file_handler.setFormatter(formatter)
            self.logger.addHandler(file_handler)

    async def process_string_with_semaphore(self, job_number: int, string: str):
        async with self.semaphore:
            result = await self.process_func(string)
            self.logger.info(f"Processed job {job_number}: {string}")
            return result

    async def process_batch(self, batch: List[str], batch_number: int, total_jobs: int):
        batch_tasks = [self.process_string_with_semaphore(i, s) for i, s in enumerate(batch)]
        batch_results = await tqdm.gather(*batch_tasks, desc=f"Batch {batch_number}")
        
        self.processed_strings.extend(batch)
        self.results.extend(batch_results)
        
        self.save_progress()
        
        completion_message = f"Batch {batch_number} completed. Total processed: {len(self.processed_strings)}/{total_jobs}"
        print(completion_message)
        self.logger.info(completion_message)

    def save_progress(self):
        with open(self.pickle_file, 'wb') as f:
            pickle.dump({'processed_strings': self.processed_strings, 'results': self.results}, f)

    async def process_strings_in_batches(self, input_strings: List[str]):
        total_jobs = len(input_strings)
        
        for i in range(0, total_jobs, self.batch_size):
            batch = input_strings[i:i+self.batch_size]
            await self.process_batch(batch, i//self.batch_size + 1, total_jobs)

        return self.processed_strings, self.results

# Example usage
async def example_process_single_string(string: str) -> str:
    sleep_time = random.uniform(2, 2.5)
    await asyncio.sleep(sleep_time)
    return f"Processed: {string} (took {sleep_time:.2f} seconds)"

async def main():
    input_strings = [f"string{i + 1}" for i in range(10)]
    
    processor = BatchProcessor(
        process_func=example_process_single_string,
        batch_size=10,
        concurrency_limit=5,
        logfile="david_test.log"
    )
    
    processed_strings, results = await processor.process_strings_in_batches(input_strings)
    print(f"All processing complete. Total processed: {len(processed_strings)}")

await main()

Batch 1: 100%|██████████| 10/10 [00:04<00:00,  2.16it/s]

Batch 1 completed. Total processed: 10/10
All processing complete. Total processed: 10


In [3]:
1000/60

16.666666666666668

In [7]:
(4300*16)/60/60

19.11111111111111